In [1]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))
getwd()

[1] "/mnt/hdd/develop/kaggle/R_sources/house_prices"

In [3]:
source('main.R')

house_prices$helpers$import_libs()

combined_dataset <- house_prices$helpers$load_data()

In [4]:
combined_dataset %>% dim
combined_dataset %>% filter(dataSource == "train") %>% dim
combined_dataset %>% filter(dataSource == "test") %>% dim

[1] 2919   82

[1] 1460   82

[1] 1459   82

In [6]:
categ_data <- 
    combined_dataset %>% 
    select(house_prices$helpers$get_character_colnames(combined_dataset))

numeric_data <- 
    combined_dataset %>% 
    select(house_prices$helpers$get_numeric_colnames(combined_dataset))

In [5]:
categ_data %>% glimpse(50)

Observations: 2,919
Variables: 45
$ Alley         <chr> NA, NA, NA, NA, NA, NA,...
$ BldgType      <chr> "1Fam", "1Fam", "1Fam",...
$ BsmtCond      <chr> "TA", "TA", "TA", "Gd",...
$ BsmtExposure  <chr> "No", "Gd", "Mn", "No",...
$ BsmtFinType1  <chr> "GLQ", "ALQ", "GLQ", "A...
$ BsmtFinType2  <chr> "Unf", "Unf", "Unf", "U...
$ BsmtQual      <chr> "Gd", "Gd", "Gd", "TA",...
$ CentralAir    <chr> "Y", "Y", "Y", "Y", "Y"...
$ Condition1    <chr> "Norm", "Feedr", "Norm"...
$ Condition2    <chr> "Norm", "Norm", "Norm",...
$ dataSource    <chr> "train", "train", "trai...
$ Electrical    <chr> "SBrkr", "SBrkr", "SBrk...
$ ExterCond     <chr> "TA", "TA", "TA", "TA",...
$ Exterior1st   <chr> "VinylSd", "MetalSd", "...
$ Exterior2nd   <chr> "VinylSd", "MetalSd", "...
$ ExterQual     <chr> "Gd", "TA", "Gd", "TA",...
$ Fence         <chr> NA, NA, NA, NA, NA, "Mn...
$ FireplaceQu   <chr> NA, "TA", "TA", "Gd", "...
$ Foundation    <chr> "PConc", "CBlock", "PCo...
$ Functional    <chr> "Typ", "Typ",

In [6]:
numeric_data %>% glimpse(50)

Observations: 2,919
Variables: 37
$ BedroomAbvGr  <int> 3, 3, 3, 3, 4, 1, 3, 3,...
$ BsmtFinSF1    <int> 706, 978, 486, 216, 655...
$ BsmtFinSF2    <int> 0, 0, 0, 0, 0, 0, 0, 32...
$ BsmtFullBath  <int> 1, 0, 1, 1, 1, 1, 1, 1,...
$ BsmtHalfBath  <int> 0, 1, 0, 0, 0, 0, 0, 0,...
$ BsmtUnfSF     <int> 150, 284, 434, 540, 490...
$ EnclosedPorch <int> 0, 0, 0, 272, 0, 0, 0, ...
$ Fireplaces    <int> 0, 1, 1, 1, 1, 0, 1, 2,...
$ FullBath      <int> 2, 2, 2, 1, 2, 1, 2, 2,...
$ GarageArea    <int> 548, 460, 608, 642, 836...
$ GarageCars    <int> 2, 2, 2, 3, 3, 2, 2, 2,...
$ GarageYrBlt   <int> 2003, 1976, 2001, 1998,...
$ GrLivArea     <int> 1710, 1262, 1786, 1717,...
$ HalfBath      <int> 1, 0, 1, 0, 1, 1, 0, 1,...
$ Id            <int> 1, 2, 3, 4, 5, 6, 7, 8,...
$ KitchenAbvGr  <int> 1, 1, 1, 1, 1, 1, 1, 1,...
$ LotArea       <int> 8450, 9600, 11250, 9550...
$ LotFrontage   <int> 65, 80, 68, 60, 84, 85,...
$ LowQualFinSF  <int> 0, 0, 0, 0, 0, 0, 0, 0,...
$ MasVnrArea    <int> 196, 0, 162, 

In [25]:
categ_data %>% select(BldgType) %>% table(useNA="ifany") -> x
freqs <- x / nrow(categ_data)

-sum(freqs * log2(freqs))

[1] 0.9659296

In [42]:
categ_data %>% 
    purrr::map(function (col) {
        freqs <- table(col, useNA="ifany") / length(col)
        -sum(freqs * log2(freqs))
    }) -> x

x %>% unlist %>% sort

Utilities        PoolQC        Street    Condition2      RoofMatl 
   0.01262749    0.03820308    0.03850369    0.10643374    0.13851938 
      Heating   MiscFeature     LandSlope    CentralAir         Alley 
   0.13977521    0.24448832    0.30379935    0.35518274    0.42339144 
   Electrical    Functional    PavedDrive    GarageCond   LandContour 
   0.50376029    0.51151268    0.52663506    0.57416377    0.62955584 
   GarageQual      BsmtCond     ExterCond      SaleType     RoofStyle 
   0.65213067    0.66945450    0.67993616    0.81047550    0.86968288 
   Condition1  BsmtFinType2      BldgType SaleCondition    dataSource 
   0.91873927    0.96490275    0.96592959    0.97189555    0.99999992 
        Fence      MSZoning      LotShape     LotConfig     ExterQual 
   1.00775062    1.04573606    1.12084385    1.20065238    1.21057907 
   MasVnrType   KitchenQual    Foundation  BsmtExposure     HeatingQC 
   1.38455666    1.42640467    1.53324046    1.56573321    1.60709360 
   GarageType      BsmtQual  GarageFinish   FireplaceQu    HouseStyle 
   1.60977228    1.65312352    1.76533212    1.79360788    1.83638249 
 BsmtFinType1   Exterior1st   Exterior2nd    MSSubClass  Neighborhood 
   2.45472429    2.72342894    2.80072036    2.92002125    4.21209953